## First, some setting up

In [1]:
:set -XOverloadedStrings -XQuasiQuotes
:set -XFlexibleContexts -XTypeApplications

In [2]:
import Control.Lens
import Data.Aeson
import Data.Aeson.Lens

import qualified Data.ByteString as BS
import Chrono.TimeStamp

## Let's go!

In [3]:
x42 = 
  "{}" & _Object . at "x" .~ Just (Integer 42)
x42

Line 2: Use ?~
Found:
_Object . at "x" .~ Just (Integer 42)
Why not:
((_Object . at "x") ?~ Integer 42)

"{\"x\":42}"

In [4]:
makeReadable bs = -- IHaskell renders Values nicely
  bs ^?! _JSON @_ @Value

makeReadable $ 
  "{}" & _Object . at "x" . below _Integer ?~ 42

{
    "x": 42
}

#### Real life data (stolen from the awesome "Optics By Example")

In [5]:
pods <- BS.readFile "pods.json"
pods & makeReadable

{
    "apiVersion": "v1",
    "items": [
        {
            "apiVersion": "v1",
            "kind": "Pod",
            "metadata": {
                "creationTimestamp": "2019-03-23T19:42:21Z",
                "labels": {
                    "name": "redis",
                    "region": "usa"
                },
                "name": "redis-h315w"
            },
            "spec": {
                "containers": [
                    {
                        "image": "redis",
                        "name": "redis",
                        "ports": [
                            {
                                "containerPort": 27017,
                                "hostPort": 27017,
                                "name": "redis",
                                "protocol": "TCP"
                            }
                        ],
                        "resources": {
                            "requests": {
                                "cpu": "100m"
                

In [6]:
pods ^.. key "items" . values . key "kind" . _String

["Pod","Pod"]

In [7]:
items = key "items" . values

pods ^? items
      . key "metadata" 
      . key "creationTimestamp"
      . _JSON @_ @TimeStamp

Just 2019-03-23T19:42:21.000000000Z

In [8]:
metadataTimestamp =
    key "metadata" 
  . key "creationTimestamp"
  . _JSON @_ @TimeStamp

mar19 = 
  read @TimeStamp "2019-03-00T00:00:00Z"

pods ^.. items
       . filteredBy ( metadataTimestamp
                    . to (compare mar19)
                    . only LT) -- those before mar19
       . key "metadata"

{
    "creationTimestamp": "2019-03-23T19:42:21Z",
    "labels": {
        "name": "redis",
        "region": "usa"
    },
    "name": "redis-h315w"
}

In [9]:
metadataRegion =
    key "metadata"
  . key "labels"
  . key "region"
  . _String

withRegion =
  pods & items
       . reindexed ( view metadataRegion ) selfIndex
      <. key "metadata" . key "labels" . key "name" . _String
     %@~ \region name -> name <> "-" <> region

withRegion & makeReadable

{
    "apiVersion": "v1",
    "items": [
        {
            "apiVersion": "v1",
            "kind": "Pod",
            "metadata": {
                "creationTimestamp": "2019-03-23T19:42:21Z",
                "labels": {
                    "name": "redis-usa",
                    "region": "usa"
                },
                "name": "redis-h315w"
            },
            "spec": {
                "containers": [
                    {
                        "image": "redis",
                        "name": "redis",
                        "ports": [
                            {
                                "containerPort": 27017,
                                "hostPort": 27017,
                                "name": "redis",
                                "protocol": "TCP"
                            }
                        ],
                        "resources": {
                            "requests": {
                                "cpu": "100m"
            